In [27]:
from pso import *
from gwo_algorithm import *
#from bat import *
from firefly_algorithm import *

In [28]:
from __future__ import division
from sklearn import svm
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score

import random
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
import math

In [29]:
data = load_digits() 

In [30]:
n_samples = len(data.images)
X = data.images.reshape((n_samples, -1))
Y = data['target']

In [31]:
# modle implementation
def fitness_function1(x):  
	clf = svm.SVC(kernel='rbf', C=x[0], gamma=x[1], random_state=42)
	scores = cross_val_score(clf, X, Y, cv=5)
	
	return scores.mean()  

In [32]:
def fitness_function2(x):
	pass

def bubble_sort(fitness_values, population):
    # sort population by ascending order of fitness values
    for i in range(len(fitness_values)):
        for j in range(0, len(fitness_values)-i-1):
            if fitness_values[j] > fitness_values[j+1] :
                fitness_values[j], fitness_values[j+1] = fitness_values[j+1], fitness_values[j]
                population[j], population[j+1] = population[j+1], population[j]
    return fitness_values, population

In [33]:
def clip_pop(pop, intervals):
    # IF BOUND IS SPECIFIED THEN CLIP 'pop' VALUES SO THAT THEY ARE IN THE SPECIFIED RANGE
    return [[random.uniform(lower_bound, upper_bound) if not lower_bound <= x <= upper_bound else x for x, (lower_bound, upper_bound) in zip(sublist, intervals)] for sublist in pop]

In [34]:
class JSI:
    
    def __init__(self, **kwargs):
        self.model = kwargs.get('model', 'svm')
        self.parameters = kwargs.get('parameters', 2)
        self.intervals = kwargs.get('intervals', [[1.0, 100.0], [0.0001, 0.1]])
        self.interval_dict = {'c': self.intervals[0], 'gamma': self.intervals[1]}
        self.Generations = kwargs.get('Generations', 10)
        self.n_pop = kwargs.get('n_pop', 10)

        if self.model == 'svm':
            self.fitness_function = fitness_function1
        elif self.model == 'decision tree':
            self.fitness_function = fitness_function2

        self.best_pop_from_all=[]
        self.best_fit_from_all=[]
        self.model_name=[]
        self.n = len(self.intervals)

        #initialize the population
        self.pop = [] 
        if self.intervals is not None:
            for i in range(self.n_pop):
                x = [random.uniform(interval[0], interval[1]) for interval in self.intervals]
                self.pop.append(x) # list of lists, for 2 dim: [[ , ], [ , ], [ , ],...,[ , ]]
            self.pop = clip_pop(self.pop, self.intervals)
        else:
            print('Please determine the intervals for the paremeters')
  
    def run(self):
        old_pop_firefly = self.pop.copy()
        old_pop_gwo = self.pop.copy()
        old_pop_pso = self.pop.copy()


        for i in range(self.Generations):
            print('iter:',i)

            best_firefly, pop_firefly, fit_firefly =firefly_algorithm(fitness_function = self.fitness_function, population =old_pop_firefly, dimensions = self.intervals, max_iter = self.Generations, alpha = 0.5, beta = 0.5, gamma = 0.5)
            print("finished with firefly")
            best, pop_GWO, fit_gwo = GWO(fitness_function = self.fitness_function, pop_size = self.n_pop, intervals = self.intervals, max_iter = self.Generations)
            print("finished with GWO")
            best_position, pop_pso, fit_pso = PSO(population = old_pop_pso, fitness_function = self.fitness_function, intervals= self.intervals, max_iter = self.Generations, c1 = 1.5, c2  =1.5)
            print("finished with PSO")

            sorted_fit_firefly, pop_firefly = bubble_sort(fit_firefly, pop_firefly)
            
            sorted_fit_gwo, pop_GWO = bubble_sort(fit_gwo, pop_GWO)
            
            sorted_fit_pso, pop_pso = bubble_sort(fit_pso, pop_pso )


            pop_firefly[self.n_pop-1] = pop_GWO[0].copy()
            pop_firefly[self.n_pop-2] = pop_pso[0].copy()

            pop_GWO[self.n_pop-1] = pop_firefly[0].copy()
            pop_GWO[self.n_pop-2] = pop_pso[0].copy()

            pop_pso[self.n_pop-1] = pop_firefly[0].copy()
            pop_pso[self.n_pop-1] = pop_GWO[0].copy()

            
            old_pop_firefly = pop_firefly
            
            old_pop_gwo = pop_GWO
           
            old_pop_pso = pop_pso
        
        # self.best_pop_from_all.append(new_pop_bat[0])
        # self.best_fit_from_all.append(sorted_fit_bat[0])
        # self.model_name.append('BAT')

        self.best_pop_from_all.append(pop_firefly[0])
        self.best_fit_from_all.append(sorted_fit_firefly[0])
        self.model_name.append('Firefly')

        self.best_pop_from_all.append(pop_GWO[0])
        self.best_fit_from_all.append(sorted_fit_gwo[0])
        self.model_name.append('GWO')

        self.best_pop_from_all.append(pop_pso[0])
        self.best_fit_from_all.append(sorted_fit_pso[0])
        self.model_name.append('PSO')


        

    def best_all(self):

        best = sorted(zip(self.best_pop_from_all, self.best_fit_from_all, self.model_name), key=lambda x: x[1], reverse=True)

        return best[0][0], best[0][1], best[0][2]
    

In [35]:
model = JSI(max_iter=10, n_pop= 10)

In [36]:
model.run()

iter: 0
finished with firefly
finished with GWO
finished with PSO
iter: 1
finished with firefly
finished with GWO
finished with PSO
iter: 2
finished with firefly
finished with GWO
finished with PSO
iter: 3
finished with firefly
finished with GWO
finished with PSO
iter: 4
finished with firefly
finished with GWO
finished with PSO
iter: 5
finished with firefly
finished with GWO
finished with PSO
iter: 6
finished with firefly
finished with GWO
finished with PSO
iter: 7
finished with firefly
finished with GWO
finished with PSO
iter: 8
finished with firefly
finished with GWO
finished with PSO
iter: 9
finished with firefly
finished with GWO
finished with PSO


In [37]:
best_para, best_score, best_model = model.best_all()
print(f'best parameters:{best_para}, best score:{best_score}, best_model:{best_model}')

best parameters:[100.0, 0.0001], best score:0.9621649644073041, best_model:GWO
